## Predictive Models

In this notebook, we will try multiple methods to predict A&E demands

In [4]:
import pandas as pd
import geopandas as gpd

In [12]:
url1 = 'https://www.opendata.nhs.scot/dataset/0d57311a-db66-4eaa-bd6d-cc622b6cbdfa/resource/a5f7ca94-c810-41b5-a7c9-25c18d43e5a4/download/weekly_ae_activity_20240714.csv'
df_week_AE = pd.read_csv(url1)

url2 = 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/022c3b27-6a58-48dc-8038-8f1f93bb0e78/download/opendata_monthly_ae_when_202405.csv'
df_month_AE = pd.read_csv(url2)

url3 = 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/c4622324-f59c-4011-a67b-83b59c59ca94/download/opendata_monthly_ae_discharge_202405.csv'
df_discharge = pd.read_csv(url3)

url4= 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/37ba17b1-c323-492c-87d5-e986aae9ab59/download/monthly_ae_activity_202405.csv'
df_month_attendances= pd.read_csv(url4)

url_hospital = 'https://www.opendata.nhs.scot/dataset/cbd1802e-0e04-4282-88eb-d7bdcfb120f0/resource/c698f450-eeed-41a0-88f7-c1e40a568acc/download/hospitals.csv'
df_hospital = pd.read_csv(url_hospital)
#df_hospital : A list of all NHS hospitals across Scotland and associated geographic information. It should be noted that this list contains all hospitals in Scotland, not only acute hospitals.


In [13]:
url_demographics= 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/6abbf8e4-e4e0-4a56-a7b9-f7c7b4171ff3/download/opendata_monthly_ae_demographics_202405.csv'
df_demographics= pd.read_csv(url_demographics)

In [8]:
shapefile_path = "SG_NHS_HealthBoards_2019"

#Reading the shapefile into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)

#Information about locations
location_df= gdf[["HBCode", "HBName"]]

In [10]:
df_month_AE.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,Day,Week,Hour,InOut,NumberOfAttendances
0,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,00:00 to 00:59,Out of Hours,20
1,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,01:00 to 01:59,Out of Hours,14
2,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,02:00 to 02:59,Out of Hours,6
3,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,03:00 to 03:59,Out of Hours,5
4,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,04:00 to 04:59,Out of Hours,5


In [15]:
df_month_attendances.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,NumberOfAttendancesAll,NumberWithin4HoursAll,NumberOver4HoursAll,PercentageWithin4HoursAll,NumberOfAttendancesEpisode,...,PercentageWithin4HoursEpisode,PercentageWithin4HoursEpisodeQF,NumberOver8HoursEpisode,NumberOver8HoursEpisodeQF,PercentageOver8HoursEpisode,PercentageOver8HoursEpisodeQF,NumberOver12HoursEpisode,NumberOver12HoursEpisodeQF,PercentageOver12HoursEpisode,PercentageOver12HoursEpisodeQF
0,200707,S92000003,S08000015,A101H,Minor Injury Unit or Other,252,252,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z
1,200707,S92000003,S08000015,A111H,Emergency Department,5414,5290,124,97.7,5414.0,...,97.7,NaN,26.0,NaN,0.5,NaN,24.0,NaN,0.4,NaN
2,200707,S92000003,S08000015,A207H,Minor Injury Unit or Other,92,92,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z
3,200707,S92000003,S08000015,A210H,Emergency Department,3530,3355,175,95.0,3530.0,...,95.0,NaN,3.0,NaN,0.1,NaN,1.0,NaN,0.0,NaN
4,200707,S92000003,S08000016,B103H,Minor Injury Unit or Other,20,20,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z


In [14]:
df_discharge.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,Age,AgeQF,Discharge,DischargeQF,NumberOfAttendances
0,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Admission to same Hospital,NaN,85
1,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Discharged Home or to usual Place of Residence,NaN,386
2,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Transferred to Other Hospital/Service,NaN,5
3,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,NaN,:,18
4,201801,S92000003,S08000015,A111H,Emergency Department,25-39,NaN,Admission to same Hospital,NaN,206


In [16]:
df_hospital.head()

,HospitalCode,HospitalName,AddressLine1,AddressLine2,AddressLine2QF,AddressLine3,AddressLine3QF,AddressLine4,AddressLine4QF,Postcode,HealthBoard,HSCP,CouncilArea,IntermediateZone,DataZone
0,A101H,Arran War Memorial Hospital,Lamlash,Isle of Arran,NaN,NaN,z,NaN,z,KA278LF,S08000015,S37000020,S12000021,S02002097,S01011176
1,A103H,Ayrshire Central Hospital,Kilwinning Road,Irvine,NaN,NaN,z,NaN,z,KA128SS,S08000015,S37000020,S12000021,S02002105,S01011213
2,A105H,Kirklandside Hospital,Kirklandside,Kilmarnock,NaN,Ayrshire,NaN,NaN,z,KA1 5LH,S08000015,S37000008,S12000008,S02001492,S01007961
3,A110H,Lady Margaret Hospital,College St,Millport,NaN,Isle of Cumbrae,NaN,NaN,z,KA280HF,S08000015,S37000020,S12000021,S02002128,S01011328
4,A111H,University Hospital Crosshouse,Kilmarnock Road,Kilmarnock,NaN,Ayrshire,NaN,NaN,z,KA2 0BE,S08000015,S37000008,S12000008,S02001504,S01008027
